# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 50 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
frames = [frm_idx for frm_idx in range(6, 26)]
y, z = (120, 160) #(y,z) is the final size of the input images

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = frames #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[0]==image.shape[1]:
                        image = image[120:240, 100:260]
                    else:
                        image = image[0:y,0:z]
                    rs_img = imresize(image, (y,z))
                    
                    batch_data[folder,idx,:,:,0] = (rs_img[:, :, 0] - np.min(rs_img[:, :, 0]))/(np.max(rs_img[:, :, 0]) - np.min(rs_img[:, :, 0]))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (rs_img[:, :, 1] - np.min(rs_img[:, :, 1]))/(np.max(rs_img[:, :, 1]) - np.min(rs_img[:, :, 1]))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (rs_img[:, :, 2] - np.min(rs_img[:, :, 2]))/(np.max(rs_img[:, :, 2]) - np.min(rs_img[:, :, 2]))#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    if image.shape[0]==image.shape[1]:
                        image = image[120:240, 100:260]
                    else:
                        image = image[0:y,0:z]
                    rs_img = imresize(image, (y,z))
                    
                    batch_data[folder,idx,:,:,0] = (rs_img[:, :, 0] - np.min(rs_img[:, :, 0]))/(np.max(rs_img[:, :, 0]) - np.min(rs_img[:, :, 0]))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (rs_img[:, :, 1] - np.min(rs_img[:, :, 1]))/(np.max(rs_img[:, :, 1]) - np.min(rs_img[:, :, 1]))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (rs_img[:, :, 2] - np.min(rs_img[:, :, 2]))/(np.max(rs_img[:, :, 2]) - np.min(rs_img[:, :, 2]))#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


In [6]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 45 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 45


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

inp_shape = [len(frames),y,z,3]
filter_shape = [2,2,2]
dense_layer_neurons = 128
dropout=0.5

model = Sequential()

model.add(Conv3D(filters = 16,
                 kernel_size=filter_shape,
                 padding='same',
                 input_shape=inp_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(.25))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#############################################################################################

model.add(Conv3D(filters = 32,
                 kernel_size=filter_shape,
                 padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(.25))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#############################################################################################

model.add(Conv3D(filters = 64,
                 kernel_size=filter_shape,
                 padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(.25))
#model.add(Conv3D(filters = 32,
#                        kernel_size=(2,2,2),
#                        activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(.25))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

############################################################################################

model.add(Conv3D(filters = 128,
                 kernel_size=filter_shape,
                 padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(.25))
#model.add(Conv3D(filters = 64,
#                        padding='same',
#                        kernel_size=(2,2,2),
#                        activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(.25))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

############################################################################################

model.add(Flatten())
model.add(Dense(dense_layer_neurons, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Dense(dense_layer_neurons, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))

# softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
from keras.optimizers import SGD, Adam # import Optimizers

optimiser = SGD() #write your optimizer

model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 20, 120, 160, 16)  400       
_________________________________________________________________
activation_1 (Activation)    (None, 20, 120, 160, 16)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 120, 160, 16)  64        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 60, 80, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 10, 60, 80, 32)    4128      
_________________________________________________________________
activation_2 (Activation)    (None, 10, 60, 80, 32)    0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 10, 60, 80, 32)    128       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 50
Source path =  Project_data/train ; batch size = 50
Epoch 1/45


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


12/14 [========================>.....] - ETA: 22s - loss: 2.3148 - categorical_accuracy: 0.2917

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


14/14 [==============================] - 150s 11s/step - loss: 2.2543 - categorical_accuracy: 0.3176 - val_loss: 1.9695 - val_categorical_accuracy: 0.3800

Epoch 00001: saving model to model_init_2020-11-0911_45_45.368416/model-00001-2.26028-0.30618-1.96947-0.38000.h5
Epoch 2/45
14/14 [==============================] - 17s 1s/step - loss: 2.2370 - categorical_accuracy: 0.3022 - val_loss: 3.4437 - val_categorical_accuracy: 0.2700

Epoch 00002: saving model to model_init_2020-11-0911_45_45.368416/model-00002-2.23702-0.30220-3.44372-0.27000.h5
Epoch 3/45
14/14 [==============================] - 19s 1s/step - loss: 2.1320 - categorical_accuracy: 0.3077 - val_loss: 1.9652 - val_categorical_accuracy: 0.3300

Epoch 00003: saving model to model_init_2020-11-0911_45_45.368416/model-00003-2.13203-0.30769-1.96525-0.33000.h5
Epoch 4/45
14/14 [==============================] - 17s 1s/step - loss: 1.5970 - categorical_accuracy: 0.3956 - val_loss: 1.2355 - val_categorical_accuracy: 0.5300

Epoch 0000

 8/14 [================>.............] - ETA: 6s - loss: 0.9518 - categorical_accuracy: 0.6538